In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'Home.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [ ]:
from basemodels.models import Tournament, Season, Team,Match
from basemodels.models import FikstureModelData
from datetime import datetime, timedelta
import pandas as pd
from django.db import IntegrityError

class SofascoreAPI:
    def __init__(self):
        pass

    def fetch_data(self):
        data = FikstureModelData.objects.all().values()
        data_list = list(data)
        df = pd.DataFrame(data_list)
        df_exploded = df.explode('data')
        expanded_data = pd.json_normalize(df_exploded['data'])
        expanded_data['tarih'] = list(df_exploded['tarih'])
        veri = expanded_data.copy()
        veri.columns = [item.replace(".", "_") for item in veri.columns]
        return veri

    def tournament_data(self):
        verim = self.fetch_data()
        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
        
        # Aynı uniqueTournament_id'ye sahip olanları tekrar kontrol et
        verim = verim.drop_duplicates("tournament_uniqueTournament_id")
        
        cols = [
            "tournament_uniqueTournament_id", "tournament_name", "tournament_slug", "tournament_category_name",
            "tournament_category_slug", "tournament_uniqueTournament_category_id", 
            "tournament_uniqueTournament_hasEventPlayerStatistics", "tournament_id", "tournament_isGroup", 
            "tournament_uniqueTournament_hasPerformanceGraphFeature"
        ]
        
        for col in cols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        verim = verim[cols]

        for _, row in verim.iterrows():
            tournament_id = row['tournament_uniqueTournament_id']
            
            if pd.isna(tournament_id):
                print(f"Geçersiz tournament_uniqueTournament_id: {tournament_id}, atlanıyor.")
                continue
        
            if not Tournament.objects.filter(tournament_id=tournament_id).exists():
                Tournament.objects.create(
                    tournament_id=row['tournament_uniqueTournament_id'],
                    tournament_name=row['tournament_name'],
                    tournament_slug=row['tournament_slug'],
                    tournament_category_name=row['tournament_category_name'],
                    tournament_category_slug=row['tournament_category_slug'],
                    tournament_uniqueTournament_category_id=row['tournament_uniqueTournament_category_id'],
                    tournament_model_id=row['tournament_id'],
                    tournament_isGroup=row['tournament_isGroup'] if pd.notna(row['tournament_isGroup']) else False,
                    tournament_uniqueTournament_hasPerformanceGraphFeature=row['tournament_uniqueTournament_hasPerformanceGraphFeature'] if pd.notna(row['tournament_uniqueTournament_hasPerformanceGraphFeature']) else False
                )
                print(f"Tournament ID {tournament_id} başarıyla eklendi.")
            else:
                print(f"Tournament ID {tournament_id} zaten mevcut, eklenmedi.")

    def season_data(self):
        verim = self.fetch_data()
        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
        
        verim = verim.drop_duplicates("season_id")
        
        season_cols = ["season_id", "season_name", "season_year","tournament_uniqueTournament_id"]

        for col in season_cols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        verim = verim[season_cols]

        for _, row in verim.iterrows():
            season_id = row['season_id']
            tournament_id = row.get('tournament_uniqueTournament_id')
    
            if pd.isna(season_id) or pd.isna(tournament_id):
                print(f"Geçersiz season_id veya tournament_id: Season ID={season_id}, Tournament ID={tournament_id}. Atlanıyor.")
                continue
    
            if not Season.objects.filter(season_id=season_id).exists():
                tournament_instance = Tournament.objects.filter(tournament_id=tournament_id).first()
                
                if tournament_instance:
                    Season.objects.create(
                        season_id=row['season_id'],
                        season_name=row['season_name'],
                        season_year=row['season_year'],
                        tournament=tournament_instance  # Doğru sütun adı burada kullanılıyor
                    )
                    print(f"Season ID {season_id} başarıyla eklendi.")
                else:
                    print(f"Tournament ID {tournament_id} bulunamadı, Season ID {season_id} eklenemedi.")
            else:
                print(f"Season ID {season_id} zaten mevcut, eklenmedi.")
    
   
    def team_data(self):

        data = self.fetch_data()
        homecols = [
                    "homeTeam_id", "homeTeam_name", "homeTeam_slug", "homeTeam_shortName",
                    "homeTeam_nameCode", "homeTeam_national", "tournament_uniqueTournament_id", "season_id"
                ]
        
        awaycols = [
                    "awayTeam_id", "awayTeam_name", "awayTeam_slug", "awayTeam_shortName",
                    "awayTeam_nameCode", "awayTeam_national", "tournament_uniqueTournament_id", "season_id"
                ]
        
        # Home takımı için veri seçimi ve kolon isimlerinin ayarlanması
        homedata = data[homecols]
        homedata.columns = ["team_id", "team_name", "team_slug", "team_shortName",
                                       "team_nameCode", "team_national", "tournament_id", "season_id"]
        
        # Away takımı için veri seçimi ve kolon isimlerinin ayarlanması
        awaydata = data[awaycols]
        awaydata.columns  = ["team_id", "team_name", "team_slug", "team_shortName",
                                       "team_nameCode", "team_national", "tournament_id", "season_id"]
        
        # Home ve Away takımları birleştirme
        teamdata = pd.concat([homedata, awaydata])
        teamdata = teamdata.drop_duplicates(subset=["team_id", "tournament_id", "season_id"])
       
        for _, row in teamdata.iterrows():
            team_id = row['team_id']
            tournament_id = row['tournament_id']
            season_id = row['season_id']
            
            # Geçersiz veya eksik id'leri atla
            if pd.isna(team_id) or pd.isna(tournament_id) or pd.isna(season_id):
                print(f"Geçersiz değer: Team ID={team_id}, Tournament ID={tournament_id}, Season ID={season_id}. Atlanıyor.")
                continue

            # İlgili tournament ve season nesnelerini getir
            tournament_instance = Tournament.objects.filter(tournament_id=tournament_id).first()
            season_instance = Season.objects.filter(season_id=season_id).first()

            try:
       
                team, created = Team.objects.get_or_create(
                    team_id=team_id,
                    defaults={
                        "team_name": row['team_name'],
                        "team_slug": row['team_slug'],
                        "team_shortName": row['team_shortName'],
                        "team_nameCode": row['team_nameCode'],
                        "team_national": row['team_national'],
                        "tournament": tournament_instance,
                        "season": season_instance,
                    }
                )
                if created:
                    print(f"Team ID {team_id} başarıyla eklendi.")
                else:
                    print(f"Team ID {team_id} zaten mevcut, ekleme yapılmadı.")
                    
            except IntegrityError:
                print(f"UNIQUE constraint hatası! Team ID {team_id} zaten veritabanında mevcut.")

    def match_data(self):
        # Fetch data from the API or existing model
        verim = self.fetch_data()
        if verim is None:
            raise ValueError("Veri işlenmedi. Lütfen önce 'process_data()' metodunu çalıştırın.")
        
     
     
        new_cols = ['id', 'startTimestamp', 'slug', 'tournament_uniqueTournament_id', 'season_id',
                    'roundInfo_round', 'status_type', 'homeTeam_id', 'awayTeam_id',
                    'homeScore_current', 'homeScore_period1', 'homeScore_period2', 'homeScore_normaltime',
                    'awayScore_current', 'awayScore_period1', 'awayScore_period2', 'awayScore_normaltime']
        
        df1 = verim[new_cols] 
        df1['date'] = pd.to_datetime(df1['startTimestamp'], unit='s') + timedelta(hours=3)
        df2 = df1.drop("startTimestamp", axis=1)
        df3 = df2.fillna(0)
        for _, row in df3.iterrows():
            match_id = row['id']
            tournament_id = row['tournament_uniqueTournament_id']
            season_id = row['season_id']
            home_team_id = row['homeTeam_id']
            away_team_id = row['awayTeam_id']
            
            # Geçersiz veya eksik id'leri atla
            if pd.isna(match_id) or pd.isna(tournament_id) or pd.isna(season_id) or pd.isna(home_team_id) or pd.isna(away_team_id):
                print(f"Geçersiz değer: Match ID={match_id}, Tournament ID={tournament_id}, Season ID={season_id}, Home Team ID={home_team_id}, Away Team ID={away_team_id}. Atlanıyor.")
                continue

            # İlgili foreign key nesnelerini getir
            tournament_instance = Tournament.objects.filter(tournament_id=tournament_id).first()
            season_instance = Season.objects.filter(season_id=season_id).first()
            home_team_instance = Team.objects.filter(team_id=home_team_id).first()
            away_team_instance = Team.objects.filter(team_id=away_team_id).first()
            try:
                if tournament_instance and season_instance and home_team_instance and away_team_instance:
                    # Match'in veritabanında olup olmadığını kontrol et, yoksa oluştur
                    match, created = Match.objects.get_or_create(
                        id=match_id,
                        defaults={
                            'startTimestamp': row['date'],  # DateTimeField olarak datetime verisi
                            'slug': row['slug'],
                            'tournament': tournament_instance,
                            'season': season_instance,
                            'homeTeam': home_team_instance,
                            'awayTeam': away_team_instance,
                            'roundInfo_round': row['roundInfo_round'],
                            'homeScore_current': row['homeScore_current'],
                            'homeScore_period1': row['homeScore_period1'],
                            'homeScore_period2': row['homeScore_period2'],
                            'homeScore_normaltime': row['homeScore_normaltime'],
                            'awayScore_current': row['awayScore_current'],
                            'awayScore_period1': row['awayScore_period1'],
                            'awayScore_period2': row['awayScore_period2'],
                            'awayScore_normaltime': row['awayScore_normaltime'],
                            'status_type': row['status_type']
                        }
                    )
                    if created:
                        print(f"Match ID {match_id} başarıyla eklendi.")
                    else:
                        print(f"Match ID {match_id} zaten mevcut, eklenmedi.")
                else:
                    print(f"Tournament ID {tournament_id}, Season ID {season_id}, Home Team ID {home_team_id}, Away Team ID {away_team_id} bulunamadı.")
            except IntegrityError:
                print(f"UNIQUE constraint hatası! Match ID {match_id} zaten veritabanında mevcut.")
            except Exception as e:
                print(f"Match ID {match_id} eklenirken bir hata oluştu: {e}")



# SofascoreAPI sınıfını kullanarak team_data'yı veritabanına kaydetmek için:
api = SofascoreAPI()
teamdata = api.fetch_data()  # Veriyi getirin (API'den ya da dosyadan)
api.tournament_data()
api.season_data()
api.team_data()
api.match_data()
print("İşlemler bitti")


Tournament ID 17 zaten mevcut, eklenmedi.
Tournament ID 8 zaten mevcut, eklenmedi.
Tournament ID 23 zaten mevcut, eklenmedi.
Tournament ID 35 zaten mevcut, eklenmedi.
Tournament ID 34 zaten mevcut, eklenmedi.
Tournament ID 37 zaten mevcut, eklenmedi.
Tournament ID 325 zaten mevcut, eklenmedi.
Tournament ID 52 zaten mevcut, eklenmedi.
Tournament ID 38 zaten mevcut, eklenmedi.
Tournament ID 185 zaten mevcut, eklenmedi.
Tournament ID 2132 zaten mevcut, eklenmedi.
Tournament ID 155 zaten mevcut, eklenmedi.
Tournament ID 11621 zaten mevcut, eklenmedi.
Tournament ID 170 zaten mevcut, eklenmedi.
Tournament ID 210 zaten mevcut, eklenmedi.
Tournament ID 18 zaten mevcut, eklenmedi.
Tournament ID 54 zaten mevcut, eklenmedi.
Tournament ID 53 zaten mevcut, eklenmedi.
Tournament ID 182 zaten mevcut, eklenmedi.
Tournament ID 390 zaten mevcut, eklenmedi.
Tournament ID 131 zaten mevcut, eklenmedi.
Tournament ID 98 zaten mevcut, eklenmedi.
Tournament ID 45 zaten mevcut, eklenmedi.
Tournament ID 266 zate

C:\Users\ahmet.yildirir\AppData\Local\Temp\ipykernel_17620\1104518459.py:183: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['date'] = pd.to_datetime(df1['startTimestamp'], unit='s') + timedelta(hours=3)


Match ID 12437039 zaten mevcut, eklenmedi.
Match ID 12437032 zaten mevcut, eklenmedi.
Match ID 12437036 zaten mevcut, eklenmedi.
Match ID 12437847 zaten mevcut, eklenmedi.
Match ID 12437835 zaten mevcut, eklenmedi.
Match ID 12437852 zaten mevcut, eklenmedi.
Match ID 12437839 zaten mevcut, eklenmedi.
Match ID 12437854 zaten mevcut, eklenmedi.
Match ID 12499331 zaten mevcut, eklenmedi.
Match ID 12499335 zaten mevcut, eklenmedi.
Match ID 12499325 zaten mevcut, eklenmedi.
Match ID 12499334 zaten mevcut, eklenmedi.
Match ID 12499329 zaten mevcut, eklenmedi.
Match ID 12499710 zaten mevcut, eklenmedi.
Match ID 12499700 zaten mevcut, eklenmedi.
Match ID 12451038 zaten mevcut, eklenmedi.
Match ID 12451034 zaten mevcut, eklenmedi.
Match ID 12451040 zaten mevcut, eklenmedi.
Match ID 12451036 zaten mevcut, eklenmedi.
Match ID 12451042 zaten mevcut, eklenmedi.
Match ID 12448385 zaten mevcut, eklenmedi.
Match ID 12448386 zaten mevcut, eklenmedi.
Match ID 12448387 zaten mevcut, eklenmedi.
Match ID 12

C:\Users\ahmet.yildirir\AppData\Local\Programs\Python\Python312\Lib\site-packages\django\db\models\fields\__init__.py:1595: RuntimeWarning: DateTimeField Match.startTimestamp received a naive datetime (2024-11-02 22:00:00) while time zone support is active.
  warnings.warn(


UNIQUE constraint hatası! Match ID 13045832 zaten veritabanında mevcut.


...

UNIQUE constraint hatası! Match ID 13045836 zaten veritabanında mevcut.


C:\Users\ahmet.yildirir\AppData\Local\Programs\Python\Python312\Lib\site-packages\django\db\models\fields\__init__.py:1595: RuntimeWarning: DateTimeField Match.startTimestamp received a naive datetime (2024-11-04 04:30:00) while time zone support is active.
  warnings.warn(


UNIQUE constraint hatası! Match ID 13045838 zaten veritabanında mevcut.
Match ID 12580610 zaten mevcut, eklenmedi.
Match ID 12580616 zaten mevcut, eklenmedi.
Match ID 12580626 zaten mevcut, eklenmedi.
Match ID 12580614 zaten mevcut, eklenmedi.
Match ID 12580629 zaten mevcut, eklenmedi.
Match ID 12580619 zaten mevcut, eklenmedi.
Match ID 12580612 zaten mevcut, eklenmedi.
Match ID 12580623 zaten mevcut, eklenmedi.
Match ID 12402282 zaten mevcut, eklenmedi.
Match ID 12402283 zaten mevcut, eklenmedi.
Match ID 12402291 zaten mevcut, eklenmedi.
Match ID 12402285 zaten mevcut, eklenmedi.
Match ID 12402288 zaten mevcut, eklenmedi.
Match ID 12402290 zaten mevcut, eklenmedi.
Match ID 12402289 zaten mevcut, eklenmedi.
Match ID 12402286 zaten mevcut, eklenmedi.
Match ID 12407832 zaten mevcut, eklenmedi.
Match ID 12407833 zaten mevcut, eklenmedi.
Match ID 12407835 zaten mevcut, eklenmedi.
Match ID 12407831 zaten mevcut, eklenmedi.
Match ID 12407834 zaten mevcut, eklenmedi.
Match ID 13029538 zaten m

C:\Users\ahmet.yildirir\AppData\Local\Programs\Python\Python312\Lib\site-packages\django\db\models\fields\__init__.py:1595: RuntimeWarning: DateTimeField Match.startTimestamp received a naive datetime (2024-11-01 23:00:00) while time zone support is active.
  warnings.warn(


Match ID 12437646 başarıyla eklendi.
